In [ ]:
# ============================================================================
# WEATHER IMPACT EMAIL NOTIFIER
# Parametric Insurance Demo — Notebook 4
# ============================================================================
#
# Architecture (two-step agent pattern):
#
#   Fabric Notebook (this file)
#       │
#       ├── Step A: Call Spark / SQL query (native)
#       │
#       └── Step B: Call Orchestrator Agent via Foundry Responses API
#                   (LLM only — enriched context + alert → professional email)
#
# Why this pattern?
#   Fabric Data Agents currently cannot be invoked as a tool by another
#   agent when the caller is a Fabric notebook session. Instead, the
#   notebook calls the Spark / SQL query (native), then passes the enriched
#   context to the Orchestrator Agent for email composition.
#
# Prerequisites:
#   - weather_alert_policy_impact.ipynb must have populated weather_impact_events
#   - An Orchestrator Agent (Foundry Responses API) must be created
#     (LLM-only — no tools/actions needed)
# ============================================================================

# 📧 Weather Impact Email Notifier

```
weather_impact_events (Delta)
        │
        ▼
  Fabric Notebook
        │
        ├──► Lakehouse SQL query
        │         │
        │◄── enriched context ──┘
        │
        ├──► Orchestrator Agent (Foundry Responses API, LLM only)
        │         │
        │    compose email using enriched context
        │         │
        │◄── email JSON ──┘
        │
        ▼
  email_notifications (Delta) + [optional webhook send]
```

| Step | Description | Component |
|------|-------------|----------|
| 0 | Config & imports | — |
| 1 | Load unnotified impact events | Spark SQL |
| 2 | Issue Spark / SQL query (native) |
| 3 | Call Orchestrator Agent to compose email | **Orchestrator Agent** (Responses API) |
| 4 | Persist emails to Delta | Spark |
| 5 | [Optional] Dispatch via webhook | HTTP |
| 6 | Mark impacts as notified | Delta merge |
| 7 | Summary dashboard | Spark SQL |

---
## 🔧 Step 0 — Configuration & Imports

In [ ]:
%pip install azure-ai-projects==2.0.0b3 azure-core --quiet

In [ ]:
import os
import json
import uuid
import time
import requests
import warnings
import textwrap
import notebookutils
from datetime import datetime, timedelta, timezone
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, TimestampType, BooleanType
)

# Foundry Responses API — for the Orchestrator Agent (LLM only)
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.identity import ClientSecretCredential

warnings.filterwarnings('ignore')

env_lib = notebookutils.variableLibrary.getLibrary("environmentVariables")
os.environ["AZURE_CLIENT_ID"]     = env_lib.AZURE_CLIENT_ID
os.environ["AZURE_TENANT_ID"]     = env_lib.AZURE_TENANT_ID
os.environ["AZURE_CLIENT_SECRET"] = env_lib.AZURE_CLIENT_SECRET

# ---------------------------------------------------------------------------
# CONFIGURATION — Edit these values for your environment
# ---------------------------------------------------------------------------

@dataclass
class EmailNotifierConfig:
    """Configuration for the weather impact email notifier."""

    # ── Foundry / AI Project (New experience — Responses API) ──────────────
    foundry_endpoint = env_lib.FOUNDRY_ENDPOINT

    # The Orchestrator Agent composes professional emails.
    # LLM-only agent (no tools) — notebook provides all context in the prompt.
    orchestrator_agent = env_lib.ORCHESTRATOR_AGENT

    # ── Processing limits ──────────────────────────────────────────────────
    max_events_per_run:      int = 10
    min_risk_score:          float = 0.3

    # ── Optional email dispatch ────────────────────────────────────────────
    email_dispatch_webhook:  str = ""

    # ── Insurance company branding ─────────────────────────────────────────
    insurer_name:            str = "Parametric Shield Insurance"
    insurer_support_email:   str = "support@parametricshield.com"
    insurer_support_phone:   str = "1-800-555-0199"
    insurer_portal_url:      str = "https://portal.parametricshield.com"


config = EmailNotifierConfig()

FOUNDRY_ENABLED = bool(
    config.foundry_endpoint and
    config.orchestrator_agent
)

DISPATCH_ENABLED = bool(config.email_dispatch_webhook)

spark    = SparkSession.builder.getOrCreate()
RUN_ID   = uuid.uuid4().hex[:8].upper()
now_utc  = datetime.now(timezone.utc).replace(tzinfo=None)

print("📧 Weather Impact Email Notifier")
print(f"   Run ID:                {RUN_ID}")
print(f"   Timestamp (UTC):       {now_utc.isoformat()}Z")
print(f"   Foundry Agent:         {'✅ ENABLED' if FOUNDRY_ENABLED else '⚠️  DISABLED (template fallback)'}")
print(f"   Email Dispatch:        {'✅ ENABLED (webhook)' if DISPATCH_ENABLED else '📋 LOCAL ONLY (Delta table)'}")
print(f"   Max events/run:        {config.max_events_per_run}")
print(f"   Min risk score:        {config.min_risk_score}")

---
## 🏗️ Setup: PolicyEmailOrchestrator + Email Schema

In [ ]:
# ============================================================================
# Policy Orchestrator Class
# ============================================================================

class PolicyEmailOrchestrator:
    """
    Fabric-native orchestration:
      1) Query Fabric Lakehouse tables
      2) Inject enrichment into prompt
      3) Call Foundry Orchestrator Agent
      4) Validate structured email output
    """

    def __init__(
        self,
        foundry_endpoint: str,
        tenant_id: str,
        client_id: str,
        client_secret: str,
        orchestrator_agent_name: str,
        insurer_config,
    ):
        self.agent_name = orchestrator_agent_name
        self.config = insurer_config

        credential = ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret,
        )

        self.project_client = AIProjectClient(
            endpoint=foundry_endpoint,
            credential=credential,
        )

        self.openai_client = self.project_client.get_openai_client()

    # ------------------------------------------------------------
    # Step 1 — Fabric enrichment
    # ------------------------------------------------------------
    def get_policy_context(self, policy_id: str) -> dict:
        df = spark.sql(f"""
            SELECT *
            FROM policies
            WHERE policy_id = '{policy_id}'
            LIMIT 1
        """)

        row = df.toPandas().iloc[0].dropna().to_dict()
        clean = {k: (v.isoformat() if hasattr(v, "isoformat") else v)
                 for k, v in row.items()}

        return json.loads(json.dumps(clean, default=str))

    # ------------------------------------------------------------
    # Step 2 — Prompt builder
    # ------------------------------------------------------------
    def _build_prompt(
        self,
        imp: Dict[str, Any],
        enrichment: Dict[str, Any],
    ) -> str:

        onset = imp.get("alert_onset")
        expires = imp.get("alert_expires")

        enriched_json = json.dumps(enrichment, indent=2)

        return f"""You are an insurance operations AI for {config.insurer_name}.

                A severe weather alert has been detected near one of our policyholders.
                Using the ENRICHED CONTEXT below (retrieved from our data systems) plus the
                ALERT DETAILS and POLICYHOLDER DETAILS, compose a professional, clear, and
                empathetic email notification.

                ENRICHED CONTEXT FROM DATA SYSTEMS:
                ────────────────────────────────────────────────────────────────────────────
                {enriched_json}
                ────────────────────────────────────────────────────────────────────────────

                The email must:
                  • Open with a warm but urgent subject line
                  • Acknowledge the specific weather threat in plain language
                  • Reference the policyholder by business name
                  • Explain exactly what triggers their parametric coverage
                  • Personalise based on their claim history (if any prior claims: reassure;
                    if first potential event: explain the automatic process clearly)
                  • List 3–4 recommended preparation steps specific to the alert type
                  • Close with contact information and a confident, supportive tone
                  • NOT promise a specific payout — only explain the coverage mechanism

                ALERT DETAILS:
                  Alert Type:    {imp['alert_event']}
                  Severity:      {imp['alert_severity']} | Urgency: {imp['alert_urgency']} | Certainty: {imp.get('alert_certainty','Unknown')}
                  Headline:      {imp.get('alert_headline', '')}
                  Affected Area: {imp.get('alert_area_desc', '')}
                  Onset:         {onset}
                  Expires:       {expires}

                POLICYHOLDER DETAILS (from weather impact event):
                  Policy ID:         {imp['policy_id']}
                  Business Name:     {imp['business_name']}
                  Business Type:     {imp['business_type']}
                  City, State:       {imp['city']}, {imp['state']}
                  Contact Email:     {imp['contact_email']}
                  Outage Threshold:  {imp['threshold_minutes']} minutes
                  Hourly Rate:       ${imp['hourly_rate']:,.2f}/hour
                  Max Payout:        ${imp['max_payout']:,.2f}
                  Distance from alert centroid: {imp['distance_km']} km
                  Risk Score:        {imp['risk_score']} (0–1 scale)

                INSURER CONTACT INFO:
                  Name:    {config.insurer_name}
                  Support: {config.insurer_support_phone} | {config.insurer_support_email}
                  Portal:  {config.insurer_portal_url}

                OUTPUT FORMAT — return a JSON object with exactly these keys:
                {{
                  "subject": "<email subject line>",
                  "body_text": "<plain-text email body>",
                  "body_html": "<HTML email body with inline styles, no external CSS>"
                }}

                Return ONLY the JSON object. No markdown fences, no preamble.
               """

    # ------------------------------------------------------------
    # Step 3 — Call Foundry Agent
    # ------------------------------------------------------------
    def _call_orchestrator(self, prompt: str) -> Dict[str, Any]:

        response = self.openai_client.responses.create(
            input=prompt,
            extra_body={
                "agent": {
                    "name": self.agent_name,
                    "type": "agent_reference",
                }
            },
        )

        text = self._extract_text(response)
        parsed = json.loads(text)

        self._validate_email(parsed)
        return parsed

    # ------------------------------------------------------------
    # Helpers
    # ------------------------------------------------------------
    def _extract_text(self, response) -> str:
        for item in response.output:
            if item.type == "message":
                return item.content[0].text.strip()
        raise RuntimeError("No message content returned")

    def _validate_email(self, email: Dict[str, Any]):

        required = ("subject", "body_text", "body_html")

        for key in required:
            if key not in email:
                raise ValueError(f"Missing key '{key}' in agent output")

    # ------------------------------------------------------------
    # Public API
    # ------------------------------------------------------------
    def generate_email(self, imp: Dict[str, Any]) -> Dict[str, str]:

        enrichment = self.get_policy_context(imp["policy_id"])
        prompt = self._build_prompt(imp, enrichment)
        return self._call_orchestrator(prompt)

# ============================================================================
# Create email_notifications Delta table if it doesn't exist
# ============================================================================

spark.sql("""
    CREATE TABLE IF NOT EXISTS email_notifications (
        notification_id   STRING,
        run_id            STRING,
        impact_id         STRING,
        policy_id         STRING,
        alert_id          STRING,
        recipient_email   STRING,
        recipient_name    STRING,
        subject           STRING,
        body_html         STRING,
        body_text         STRING,
        generation_method STRING,
        dispatch_status   STRING,
        dispatch_error    STRING,
        created_at        TIMESTAMP,
        dispatched_at     TIMESTAMP
    ) USING DELTA
""")

print("✅ email_notifications table ready.")

---
## 📥 Step 1 — Load Unnotified Impact Events

In [ ]:
print("📥 Loading unnotified weather impact events from lakehouse...")

try:
    pending_df = spark.sql(f"""
        SELECT *
        FROM   weather_impact_events
        WHERE  impact_status = 'pending_notification'
          AND  risk_score   >= {config.min_risk_score}
        ORDER BY risk_score DESC, created_at DESC
        LIMIT  {config.max_events_per_run}
    """)
    pending = [row.asDict() for row in pending_df.collect()]
    print(f"  Pending impact events to process: {len(pending)}")
except Exception as e:
    print(f"  ❌ Could not query weather_impact_events: {e}")
    print("     Ensure weather_alert_policy_impact.ipynb has been run first.")
    pending = []

if not pending:
    print("\nℹ️  No pending impact events. Either:")
    print("   • Run weather_alert_policy_impact.ipynb first to populate impact events, or")
    print("   • All current events have already been notified.")

if pending:
    display(pending_df.select(
        "impact_id", "policy_id", "business_name", "city", "state",
        "alert_event", "alert_severity", "risk_score", "contact_email"
    ))

---
## 🤖 Steps 2–3 — Two-Step Agent Pattern

For each pending impact event, the **notebook** orchestrates two dependent calls:

1. **Step 2 — Bootstrap Fabric Data Agent** :
   - Configures the FabricDataAgentClient (so that it can be used by PolicyEmailOrchestrator)

2. **Step 3 — PolicyEmailOrchestrator** :
   PolicyEmailOrchestrator
      │
      ├── FabricDataAgentClient  → policy enrichment
      │
      └── Foundry Orchestrator Agent → email generation

**Why not agent-to-agent?** Fabric Data Agents currently cannot be invoked
as a tool by another agent when the caller is a Fabric notebook session.
This two-step pattern gives us full control and visibility at each stage.

In [ ]:
# ============================================================================
# Fallback: rule-based context (when Foundry is unavailable)
# ============================================================================

def _build_fallback_context(imp: Dict[str, Any]) -> str:
    """Build a minimal context string from the impact event data itself."""
    return (
        f"Policy {imp['policy_id']} — {imp['business_name']} "
        f"({imp['business_type']}) in {imp['city']}, {imp['state']}. "
        f"Outage threshold: {imp['threshold_minutes']} min, "
        f"Hourly rate: ${imp['hourly_rate']:,.2f}, "
        f"Max payout: ${imp['max_payout']:,.2f}. "
        f"No additional claim history available (data agent unavailable)."
    )

# ============================================================================
# Fallback: rule-based email template (when Foundry is unavailable)
# ============================================================================

def generate_template_email(imp: Dict[str, Any]) -> Dict[str, str]:
    """
    Generate a professional policy impact notification email from a
    hard-coded template. Used as fallback when Foundry is not configured.
    """
    onset_str   = imp.get("alert_onset").strftime("%B %d, %Y at %I:%M %p UTC") if imp.get("alert_onset") else "Immediately"
    expires_str = imp.get("alert_expires").strftime("%B %d, %Y at %I:%M %p UTC") if imp.get("alert_expires") else "Until further notice"

    subject = (
        f"[{config.insurer_name}] Weather Alert — {imp['alert_event']} "
        f"near {imp['city']}, {imp['state']} | Policy {imp['policy_id']}"
    )

    body_text = textwrap.dedent(f"""
        Dear {imp['business_name']},

        This is an important notification from {config.insurer_name} regarding your
        Business Interruption Insurance policy ({imp['policy_id']}).

        A {imp['alert_severity']} weather alert has been issued in your area that
        may impact your business operations.

        ALERT DETAILS
        ─────────────────────────────────────────────────────────────────
        Alert Type:   {imp['alert_event']}
        Severity:     {imp['alert_severity']}
        Urgency:      {imp['alert_urgency']}
        Affected Area:{imp['alert_area_desc']}
        Onset:        {onset_str}
        Expires:      {expires_str}
        Distance:     Approximately {imp['distance_km']} km from your registered location

        WHAT THIS MEANS FOR YOUR POLICY
        ─────────────────────────────────────────────────────────────────
        Your policy is designed to respond automatically to qualifying power
        outages. If this weather event causes an outage that exceeds your
        threshold of {imp['threshold_minutes']} minutes, a claim will be
        processed automatically — no action is required from you.

        Your coverage summary:
          • Policy ID:          {imp['policy_id']}
          • Outage Threshold:   {imp['threshold_minutes']} minutes
          • Hourly Rate:        ${imp['hourly_rate']:,.2f}/hour
          • Maximum Payout:     ${imp['max_payout']:,.2f}

        RECOMMENDED STEPS
        ─────────────────────────────────────────────────────────────────
        1. Review the NWS alert at https://alerts.weather.gov
        2. Ensure your contact information is current in our portal
        3. Document any business disruptions for your records
        4. Claims are processed automatically — no claim form needed

        CONTACT US
        ─────────────────────────────────────────────────────────────────
        Phone:  {config.insurer_support_phone}
        Email:  {config.insurer_support_email}
        Portal: {config.insurer_portal_url}

        This notification was generated automatically based on active NWS
        weather alerts and your registered policy location.

        Sincerely,
        {config.insurer_name} — Risk Monitoring Team
    """).strip()

    body_html = (
        "<html><body style='font-family:Arial,sans-serif;max-width:680px;margin:auto;'>"
        + body_text.replace("\n", "<br>")
        + "</body></html>"
    )

    return {"subject": subject, "body_text": body_text, "body_html": body_html}


# ============================================================================
# Combined: two-step agent pipeline for a single impact event
# ============================================================================

def generate_email_for_impact(imp: Dict[str, Any]) -> tuple:
    """
    Two-step agent pipeline:
      1. Fabric Data Agent (direct) → enriched context
      2. Orchestrator Agent (LLM only) → email JSON

    Returns (email_dict, generation_method)
    """
    # Step 2: Query Fabric Data Agent for enriched context
    if FOUNDRY_ENABLED:
        print(f"  📊 Step 2 — Bootstrapping the PolicyEmailOrchestrator...")

        orchestrator = PolicyEmailOrchestrator(
            foundry_endpoint=config.foundry_endpoint,
            tenant_id=env_lib.AZURE_TENANT_ID,
            client_id=env_lib.AZURE_CLIENT_ID,
            client_secret=env_lib.AZURE_CLIENT_SECRET,
            orchestrator_agent_name=env_lib.ORCHESTRATOR_AGENT,
            insurer_config=config,
        )
    else:
        enriched_context = _build_fallback_context(imp)

    # Step 3: Compose email with Orchestrator Agent
    if FOUNDRY_ENABLED and orchestrator:
        print(f"  ✉️  Step 3 — Composing email with Orchestrator Agent...")
        email = orchestrator.generate_email(imp)
        method = (
            "notebook→Spark/SQL query (native)→orchestrator"
        )
    else:
        email = generate_template_email(imp)
        method = "rule_based_template"

    return email, method


print("✅ Email generation helpers ready.")
print("   Pattern: Notebook → Spark / SQL query (native) → Orchestrator Agent (LLM only)")

In [ ]:
# ============================================================================
# Main loop: process each pending impact event
# ============================================================================

if not pending:
    print("ℹ️  No pending impact events to process. Skipping email generation.")
else:
    mode_desc = (
        "Notebook → Spark / SQL query (native) → Orchestrator Agent"
        if FOUNDRY_ENABLED
        else "Rule-based template"
    )
    print(f"🤖 Processing {len(pending)} impact event(s)...")
    print(f"   Using: {mode_desc}\n")

    notification_records: List[Dict[str, Any]] = []

    for i, imp in enumerate(pending, 1):
        print(f"[{i}/{len(pending)}] {imp['business_name']} ({imp['city']}, {imp['state']}) — {imp['alert_event']}")

        email, generation_method = generate_email_for_impact(imp)

        notification_records.append({
            "notification_id":   f"NOTIF-{RUN_ID}-{uuid.uuid4().hex[:6].upper()}",
            "run_id":            RUN_ID,
            "impact_id":         imp["impact_id"],
            "policy_id":         imp["policy_id"],
            "alert_id":          imp["alert_id"],
            "recipient_email":   imp["contact_email"],
            "recipient_name":    imp["business_name"],
            "subject":           email["subject"],
            "body_html":         email["body_html"],
            "body_text":         email["body_text"],
            "generation_method": generation_method,
            "dispatch_status":   "pending",
            "dispatch_error":    None,
            "created_at":        now_utc,
            "dispatched_at":     None,
        })

        # Brief pause to avoid rate-limiting on the Foundry endpoint
        if FOUNDRY_ENABLED and i < len(pending):
            time.sleep(1)

    print(f"\n  ✅ Generated {len(notification_records)} email(s).")

---
## 💾 Step 4 — Persist Emails to Delta

In [ ]:
if not pending:
    print("ℹ️  Nothing to save.")
else:
    notif_schema = StructType([
        StructField("notification_id",   StringType()),
        StructField("run_id",            StringType()),
        StructField("impact_id",         StringType()),
        StructField("policy_id",         StringType()),
        StructField("alert_id",          StringType()),
        StructField("recipient_email",   StringType()),
        StructField("recipient_name",    StringType()),
        StructField("subject",           StringType()),
        StructField("body_html",         StringType()),
        StructField("body_text",         StringType()),
        StructField("generation_method", StringType()),
        StructField("dispatch_status",   StringType()),
        StructField("dispatch_error",    StringType()),
        StructField("created_at",        TimestampType()),
        StructField("dispatched_at",     TimestampType()),
    ])

    notif_df = spark.createDataFrame(notification_records, schema=notif_schema)
    notif_df.write.format("delta").mode("append").saveAsTable("email_notifications")
    print(f"✅ Saved {len(notification_records)} email record(s) to Delta table: email_notifications")

---
## 🚀 Step 5 — [Optional] Dispatch Emails via Webhook

In [ ]:
# ============================================================================
# Optional: POST email payloads to a Logic App / Power Automate webhook
# or any HTTP endpoint that accepts JSON and dispatches emails.
#
# Expected webhook payload shape:
#   {
#     "to":      "recipient@example.com",
#     "subject": "...",
#     "html":    "<html>...",
#     "text":    "...",
#     "from":    "noreply@parametricshield.com",
#     "metadata": { "notification_id": "...", "policy_id": "...", "impact_id": "..." }
#   }
# ============================================================================

if not pending:
    print("ℹ️  Nothing to dispatch.")
elif not DISPATCH_ENABLED:
    print("ℹ️  Email dispatch webhook not configured.")
    print("   Set config.email_dispatch_webhook to a Logic App / Power Automate URL to send emails.")
    print("   Emails are saved to the email_notifications Delta table for review.")
else:
    print(f"🚀 Dispatching {len(notification_records)} email(s) via webhook...\n")
    dispatched, dispatch_failed = 0, 0

    for notif in notification_records:
        payload = {
            "to":      notif["recipient_email"],
            "subject": notif["subject"],
            "html":    notif["body_html"],
            "text":    notif["body_text"],
            "from":    f"noreply@{config.insurer_support_email.split('@')[1]}",
            "metadata": {
                "notification_id": notif["notification_id"],
                "policy_id":       notif["policy_id"],
                "impact_id":       notif["impact_id"],
                "run_id":          notif["run_id"],
            }
        }
        try:
            resp = requests.post(
                config.email_dispatch_webhook,
                json=payload,
                timeout=15,
            )
            resp.raise_for_status()
            notif["dispatch_status"] = "dispatched"
            notif["dispatched_at"]   = datetime.now(timezone.utc).replace(tzinfo=None)
            dispatched += 1
            print(f"  ✅ Dispatched → {notif['recipient_email']} ({notif['policy_id']})")
        except Exception as e:
            notif["dispatch_status"] = "failed"
            notif["dispatch_error"]  = str(e)[:500]
            dispatch_failed += 1
            print(f"  ❌ Failed    → {notif['recipient_email']} ({notif['policy_id']}): {e}")

    print(f"\n  Dispatched: {dispatched}  |  Failed: {dispatch_failed}")

    # Update dispatch status in Delta
    if dispatched or dispatch_failed:
        update_schema = StructType([
            StructField("notification_id", StringType()),
            StructField("dispatch_status", StringType()),
            StructField("dispatch_error",  StringType()),
            StructField("dispatched_at",   TimestampType()),
        ])
        update_df = spark.createDataFrame(
            [{"notification_id": n["notification_id"],
              "dispatch_status": n["dispatch_status"],
              "dispatch_error":  n.get("dispatch_error"),
              "dispatched_at":   n.get("dispatched_at")} for n in notification_records],
            schema=update_schema,
        )
        update_df.createOrReplaceTempView("dispatch_updates")
        spark.sql("""
            MERGE INTO email_notifications AS target
            USING dispatch_updates AS source
            ON target.notification_id = source.notification_id
            WHEN MATCHED THEN UPDATE SET
                target.dispatch_status = source.dispatch_status,
                target.dispatch_error  = source.dispatch_error,
                target.dispatched_at   = source.dispatched_at
        """)
        print("✅ Dispatch status updated in email_notifications.")

---
## ✅ Step 6 — Mark Impact Events as Notified

In [ ]:
if not pending:
    print("ℹ️  Nothing to mark.")
else:
    processed_impact_ids = [imp["impact_id"] for imp in pending]
    ids_literal = ", ".join(f"'{iid}'" for iid in processed_impact_ids)

    try:
        spark.sql(f"""
            UPDATE weather_impact_events
            SET    impact_status = 'notified'
            WHERE  impact_id IN ({ids_literal})
        """)
        print(f"✅ Marked {len(processed_impact_ids)} impact event(s) as 'notified' in weather_impact_events.")
    except Exception as e:
        print(f"⚠️  Could not update impact_status: {e}")
        print("   Impact events remain in 'pending_notification' status — they will be reprocessed next run.")

---
## 📊 Step 7 — Summary Dashboard

In [ ]:
print("=" * 70)
print(f"RUN SUMMARY — Run ID: {RUN_ID}")
print("=" * 70)

mode_label = (
    "Notebook → Fabric Data Agent (SDK) → Orchestrator Agent"
    if FOUNDRY_ENABLED and FABRIC_DATA_AGENT_ENABLED
    else "Notebook → Orchestrator Agent"
    if FOUNDRY_ENABLED
    else "Rule-based template"
)
print(f"  Email generation: {mode_label}")
print(f"  Impact events processed: {len(pending)}")

if pending:
    gen_counts = {}
    for n in notification_records:
        m = n["generation_method"]
        gen_counts[m] = gen_counts.get(m, 0) + 1
    for method, cnt in gen_counts.items():
        print(f"  Generated via {method}: {cnt}")

print()
print("--- Email Notifications (This Run) ---")
try:
    display(spark.sql(f"""
        SELECT
            notification_id,
            policy_id,
            recipient_name,
            recipient_email,
            subject,
            generation_method,
            dispatch_status,
            created_at
        FROM email_notifications
        WHERE run_id = '{RUN_ID}'
        ORDER BY created_at
    """))
except Exception as e:
    print(f"  Could not display table: {e}")

print()
print("--- Email Notifications (All Time) ---")
try:
    display(spark.sql("""
        SELECT
            generation_method,
            dispatch_status,
            COUNT(*) AS total,
            MIN(created_at) AS first_sent,
            MAX(created_at) AS last_sent
        FROM email_notifications
        GROUP BY generation_method, dispatch_status
        ORDER BY total DESC
    """))
except Exception:
    pass

print()
print("--- Sample Email Preview (most recent) ---")
try:
    sample = spark.sql("""
        SELECT subject, body_text
        FROM   email_notifications
        ORDER BY created_at DESC
        LIMIT  1
    """).collect()
    if sample:
        print(f"\nSUBJECT: {sample[0]['subject']}\n")
        body_lines = (sample[0]['body_text'] or "").split("\n")
        for line in body_lines[:60]:
            print(line)
        if len(body_lines) > 60:
            print(f"... [{len(body_lines) - 60} more lines]")
except Exception as e:
    print(f"  Could not preview email: {e}")

# Refresh SQL Lakehouse tables
schema_name = "dbo"
tables_df = spark.sql(f"SHOW TABLES IN {schema_name}")
tables = [row.tableName for row in tables_df.collect()]

for table in tables:
    spark.sql(f"REFRESH TABLE {schema_name}.{table}")
print(f"Metadata refresh completed for schema: {schema_name}")

print()
print("✅ Notebook complete.")
print("   📋 Emails are stored in the email_notifications Delta table.")
print("   🚀 To send emails, configure config.email_dispatch_webhook with a Logic App URL.")